# Keypoint augmentation

In [ ]:
! unzip -q "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition/pose_dataset.zip"

In [ ]:
import cv2
import numpy as np
from glob import glob
from tqdm.auto import tqdm
import os

In [ ]:
source_dir = "pose_dataset"
dist_dir = "pose_dataset_aug"

In [ ]:
all_keypoints_paths = glob(f"{source_dir}/*/*/*.npy")
len(all_keypoints_paths)

22260

In [ ]:
def get_dist_path(sourch_path, aug_type, extra_arg=''):
    dist_path = sourch_path.replace(source_dir, dist_dir)
    dist_path = dist_path.split('/')
    dist_path[-2] = f"{dist_path[-2]}_{aug_type}_{extra_arg}"
    return '/'.join(dist_path)

In [ ]:
height, width = (480, 640)
def rotate_image_and_keypoints(keypoints, angle):
    # Rotate the image
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    # Rotate the keypoints
    rotated_keypoints = cv2.transform(keypoints.reshape((1, -1, 2)), rotation_matrix)
    return rotated_keypoints.reshape(-1, 17, 2)

def zoom_image_and_keypoints(keypoints, scale):
    return keypoints * scale

def flip_image_and_keypoints(keypoints):
    # Flip the keypoints
    flipped_keypoints = keypoints.copy()
    flipped_keypoints[:, :, 0] = width - flipped_keypoints[:, :, 0]
    return flipped_keypoints

In [ ]:
for keypoint_path in tqdm(all_keypoints_paths):
    keypoints = np.load(keypoint_path)
    for angle in [30,-30,20,-20,10,-10]:
        dist_path = get_dist_path(keypoint_path, 'rotation', angle)
        rotated_data = rotate_image_and_keypoints(keypoints[:,:,:2], angle)
        resulted_data = np.zeros(keypoints.shape)

        resulted_data[:,:,:2] = rotated_data
        resulted_data[:,:,2] = keypoints[:,:,2]

        os.makedirs(os.path.dirname(dist_path), exist_ok=True)
        np.save(dist_path, resulted_data)

    for scale in [1.1,1.2,1.3,0.9,0.8,0.7]:
        dist_path = get_dist_path(keypoint_path, 'scale', scale)
        resulted_data = zoom_image_and_keypoints(keypoints, scale)
        os.makedirs(os.path.dirname(dist_path), exist_ok=True)
        np.save(dist_path, resulted_data)

    dist_path = get_dist_path(keypoint_path, 'flip')
    resulted_data = flip_image_and_keypoints(keypoints)
    os.makedirs(os.path.dirname(dist_path), exist_ok=True)
    np.save(dist_path, resulted_data)

  0%|          | 0/22260 [00:00<?, ?it/s]

In [ ]:
import shutil
for keypoint_path in tqdm(all_keypoints_paths):
    dist_path = keypoint_path.replace(source_dir, dist_dir)
    os.makedirs(os.path.dirname(dist_path), exist_ok=True)
    shutil.copyfile(keypoint_path, dist_path)

  0%|          | 0/22260 [00:00<?, ?it/s]

In [ ]:
for old_path in tqdm(glob("/content/pose_dataset_aug/*/*/*")):
    new_path = old_path.replace('.jpg', '')
    os.rename(old_path, new_path)

  0%|          | 0/311640 [00:00<?, ?it/s]

In [ ]:
! zip -q -r pose_dataset_aug.zip pose_dataset_aug

In [ ]:
! cp pose_dataset_aug.zip "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition"

# Binary img augmentation

In [ ]:
! unzip -q "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition/dataset_segmentation_binary.zip"
# ! unzip -q "/content/drive/MyDrive/Colab Notebooks/ActionDetection/17_11_23/Datasets/aug_seg_rgb.zip"

In [ ]:
import cv2
import numpy as np
from glob import glob
from tqdm.auto import tqdm
import os
import shutil
from skimage import img_as_ubyte
from skimage.transform import resize
from PIL import Image

## fill the deleted images

In [ ]:
# def fill_none_with_closest(lst):
#     n = len(lst)

#     for i in range(n):
#         if lst[i] is None:
#             # Look for the closest non-None value before
#             left = i - 1
#             while left >= 0 and lst[left] is None:
#                 left -= 1

#             # Look for the closest non-None value after
#             right = i + 1
#             while right < n and lst[right] is None:
#                 right += 1

#             # Fill the None with the closest found values
#             if left >= 0:
#                 lst[i] = lst[left]
#             if right < n:
#                 lst[i] = lst[right]

#     return lst


# import shutil



# for dir in glob(f"{source_dir}/*/*"):
#     total_files = len(glob(f"{dir}/*"))
#     if total_files < 20:
#         all_img_paths = glob(f"{dir}/*")
#         img_indexs = [None]*20
#         for img_path in all_img_paths:
#             index = int(img_path.split('/')[-1].split('.')[0])
#             img_indexs[index] =  img_path
#         img_indexs = fill_none_with_closest(img_indexs)
#         for index, img_index in enumerate(img_indexs):
#             if img_index.split('/')[-1] != f"{index:0>2}.jpg":
#                 shutil.copyfile(img_index,
#                                 img_index.replace(f"{img_index.split('/')[-1]}", f"{index:0>2}.jpg")
#                                 )


In [ ]:
IMG_SIZE_H = 480
IMG_SIZE_W = 640
def process_image(img):
    h, w = img.shape[:2]
    pad_h = max(0, (IMG_SIZE_H - h) // 2)
    pad_w = max(0, (IMG_SIZE_W - w) // 2)
    img = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode='constant', constant_values=0)
    crop_h = max(0, (h-IMG_SIZE_H) // 2)
    crop_w = max(0, (w-IMG_SIZE_W) // 2)
    img = img[crop_h:IMG_SIZE_H+crop_h, crop_w:IMG_SIZE_W+crop_w, :]
    return img

def zoom_image(image, scale):
    # Resize the image
    height, width = image.shape[:2]
    new_height, new_width = int(height * scale), int(width * scale)
    zoomed_image = cv2.resize(image, (new_width, new_height))
    return process_image(zoomed_image)

In [ ]:
IMG_SIZE_H = 480
IMG_SIZE_W = 640

def rotate_image(image, angle):
    # Rotate the image
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

def process_image(img):
    h, w = img.shape[:2]
    pad_h = max(0, (IMG_SIZE_H - h) // 2)
    pad_w = max(0, (IMG_SIZE_W - w) // 2)
    img = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode='constant', constant_values=0)
    crop_h = max(0, (h-IMG_SIZE_H) // 2)
    crop_w = max(0, (w-IMG_SIZE_W) // 2)
    img = img[crop_h:IMG_SIZE_H+crop_h, crop_w:IMG_SIZE_W+crop_w, :]
    return img

def zoom_image(image, scale):
    # Resize the image
    height, width = image.shape[:2]
    new_height, new_width = int(height * scale), int(width * scale)
    zoomed_image = cv2.resize(image, (new_width, new_height))
    return process_image(zoomed_image)

def flip_image(image):
    flipped_image = cv2.flip(image, 1)
    return flipped_image

# def adjust_brightness(image, factor):
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     hsv_image[:,:,2] = np.clip(hsv_image[:,:,2] * factor, 0, 255).astype(np.uint8)
#     brightness_adjusted_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
#     return brightness_adjusted_image

In [ ]:
def get_dist_path(sourch_path, aug_type, extra_arg=''):
    dist_path = sourch_path.replace(source_dir, dist_dir)
    dist_path = dist_path.split('/')
    dist_path[-2] = f"{dist_path[-2]}_{aug_type}_{extra_arg}"
    return '/'.join(dist_path)

In [ ]:
source_dir = "dataset_segmentation_binary"
dist_dir = "aug_seg_bin"

In [ ]:
all_img_paths = glob(f"{source_dir}/*/*/*")
len(all_img_paths)

22260

In [ ]:
for img_path in tqdm(all_img_paths):
    img = cv2.imread(img_path)
    for angle in [30,-30,20,-20,10,-10]:
        dist_path = get_dist_path(img_path, 'rotation', angle)
        resulted_data = rotate_image(img, angle)
        os.makedirs(os.path.dirname(dist_path), exist_ok=True)
        cv2.imwrite(dist_path, resulted_data)

    for scale in [1.1,1.2,1.3,0.9,0.8,0.7]:
        dist_path = get_dist_path(img_path, 'scale', scale)
        resulted_data = zoom_image(img, scale)
        os.makedirs(os.path.dirname(dist_path), exist_ok=True)
        cv2.imwrite(dist_path, resulted_data)

    dist_path = get_dist_path(img_path, 'flip')
    resulted_data = flip_image(img)
    os.makedirs(os.path.dirname(dist_path), exist_ok=True)
    cv2.imwrite(dist_path, resulted_data)

  0%|          | 0/22260 [00:00<?, ?it/s]

In [ ]:
import shutil
for img_path in tqdm(all_img_paths):
    dist_path = img_path.replace(source_dir, dist_dir)
    os.makedirs(os.path.dirname(dist_path), exist_ok=True)
    shutil.copyfile(img_path, dist_path)

  0%|          | 0/22260 [00:00<?, ?it/s]

In [ ]:
! zip -q -r "aug_seg_bin.zip" "aug_seg_bin"

In [ ]:
! cp "aug_seg_bin.zip" "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition"

# RGB img augmentation

In [ ]:
! unzip -q "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition/dataset_segmentation_rgb.zip"

In [ ]:
import cv2
import numpy as np
from glob import glob
from tqdm.auto import tqdm
import os
import shutil
from skimage import img_as_ubyte
from skimage.transform import resize
from PIL import Image

In [ ]:
source_dir = "dataset_segmentation_rgb"
dist_dir = "aug_seg_rgb"

In [ ]:
all_img_paths = glob(f"{source_dir}/*/*/*")
len(all_img_paths)

21994

In [ ]:
def fill_none_with_closest(lst):
    n = len(lst)

    for i in range(n):
        if lst[i] is None:
            # Look for the closest non-None value before
            left = i - 1
            while left >= 0 and lst[left] is None:
                left -= 1

            # Look for the closest non-None value after
            right = i + 1
            while right < n and lst[right] is None:
                right += 1

            # Fill the None with the closest found values
            if left >= 0:
                lst[i] = lst[left]
            if right < n:
                lst[i] = lst[right]

    return lst



for dir in glob(f"{source_dir}/*/*"):
    total_files = len(glob(f"{dir}/*"))
    if total_files < 20:
        all_img_paths = glob(f"{dir}/*")
        img_indexs = [None]*20
        for img_path in all_img_paths:
            index = int(img_path.split('/')[-1].split('.')[0])
            img_indexs[index] =  img_path
        img_indexs = fill_none_with_closest(img_indexs)
        for index, img_index in enumerate(img_indexs):
            if img_index.split('/')[-1] != f"{index:0>2}.jpg":
                shutil.copyfile(img_index,
                                img_index.replace(f"{img_index.split('/')[-1]}", f"{index:0>2}.jpg")
                                )


In [ ]:
all_img_paths = glob(f"{source_dir}/*/*/*")
len(all_img_paths)

22260

In [ ]:
def get_dist_path(sourch_path, aug_type, extra_arg=''):
    dist_path = sourch_path.replace(source_dir, dist_dir)
    dist_path = dist_path.split('/')
    dist_path[-2] = f"{dist_path[-2]}_{aug_type}_{extra_arg}"
    return '/'.join(dist_path)

In [ ]:
IMG_SIZE_H = 480
IMG_SIZE_W = 640
def process_image(img):
    h, w = img.shape[:2]
    pad_h = max(0, (IMG_SIZE_H - h) // 2)
    pad_w = max(0, (IMG_SIZE_W - w) // 2)
    img = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode='constant', constant_values=0)
    crop_h = max(0, (h-IMG_SIZE_H) // 2)
    crop_w = max(0, (w-IMG_SIZE_W) // 2)
    img = img[crop_h:IMG_SIZE_H+crop_h, crop_w:IMG_SIZE_W+crop_w, :]
    return img

def zoom_image(image, scale):
    # Resize the image
    height, width = image.shape[:2]
    new_height, new_width = int(height * scale), int(width * scale)
    zoomed_image = cv2.resize(image, (new_width, new_height))
    return process_image(zoomed_image)

In [ ]:
IMG_SIZE_H = 480
IMG_SIZE_W = 640

def rotate_image(image, angle):
    # Rotate the image
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

def process_image(img):
    h, w = img.shape[:2]
    pad_h = max(0, (IMG_SIZE_H - h) // 2)
    pad_w = max(0, (IMG_SIZE_W - w) // 2)
    img = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode='constant', constant_values=0)
    crop_h = max(0, (h-IMG_SIZE_H) // 2)
    crop_w = max(0, (w-IMG_SIZE_W) // 2)
    img = img[crop_h:IMG_SIZE_H+crop_h, crop_w:IMG_SIZE_W+crop_w, :]
    return img

def zoom_image(image, scale):
    # Resize the image
    height, width = image.shape[:2]
    new_height, new_width = int(height * scale), int(width * scale)
    zoomed_image = cv2.resize(image, (new_width, new_height))
    return process_image(zoomed_image)

def flip_image(image):
    flipped_image = cv2.flip(image, 1)
    return flipped_image

# def adjust_brightness(image, factor):
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     hsv_image[:,:,2] = np.clip(hsv_image[:,:,2] * factor, 0, 255).astype(np.uint8)
#     brightness_adjusted_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
#     return brightness_adjusted_image

In [ ]:
for img_path in tqdm(all_img_paths):
    img = cv2.imread(img_path)
    for angle in [30,-30,20,-20,10,-10]:
        dist_path = get_dist_path(img_path, 'rotation', angle)
        resulted_data = rotate_image(img, angle)
        os.makedirs(os.path.dirname(dist_path), exist_ok=True)
        cv2.imwrite(dist_path, resulted_data)

    for scale in [1.1,1.2,1.3,0.9,0.8,0.7]:
        dist_path = get_dist_path(img_path, 'scale', scale)
        resulted_data = zoom_image(img, scale)
        os.makedirs(os.path.dirname(dist_path), exist_ok=True)
        cv2.imwrite(dist_path, resulted_data)

    dist_path = get_dist_path(img_path, 'flip')
    resulted_data = flip_image(img)
    os.makedirs(os.path.dirname(dist_path), exist_ok=True)
    cv2.imwrite(dist_path, resulted_data)

  0%|          | 0/22260 [00:00<?, ?it/s]

In [ ]:
import shutil
for img_path in tqdm(all_img_paths):
    dist_path = img_path.replace(source_dir, dist_dir)
    os.makedirs(os.path.dirname(dist_path), exist_ok=True)
    shutil.copyfile(img_path, dist_path)

  0%|          | 0/22260 [00:00<?, ?it/s]

In [ ]:
! zip -q -r "aug_seg_rgb.zip" "aug_seg_rgb"

In [ ]:
! cp "aug_seg_rgb.zip" "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition"